In [ ]:
"""

Author: Annam.ai IIT Ropar
Team Name: Ice 'N' Dagger
Team Members: Barun Saha, Bibaswan Das
Leaderboard Rank: 70

"""

# This is the notebook used for making the inferences using the model trained.



In [1]:
!pip install -q torch torchvision timm pandas scikit-learn joblib

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
import timm
import joblib

from preprocessing import SoilDataset, test_transform

# Constants
BATCH_SIZE = 32
IMG_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "swin_base_patch4_window7_224"
TEST_DIR = "/kaggle/input/soil-classification/soil_classification-2025/test"
TEST_CSV = "/kaggle/input/soil-classification/soil_classification-2025/test_ids.csv"
BEST_MODEL_PATH = "best_model.pth"

# Load test data
test_df = pd.read_csv(TEST_CSV)
label_encoder = joblib.load("label_encoder.pkl")
test_dataset = SoilDataset(test_df, TEST_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Model
model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=len(label_encoder.classes_))
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.to(DEVICE)
model.eval()

# Inference
predictions = []
image_ids = []
with torch.no_grad():
    for images, _, ids in test_loader:
        images = images.to(DEVICE)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        predictions.extend(preds.cpu().numpy())
        image_ids.extend(ids)

# Save results
submission = pd.DataFrame({
    'image_id': image_ids,
    'soil_type': label_encoder.inverse_transform(predictions)
})
submission.to_csv("submission.csv", index=False)
print("Saved predictions to submission.csv")